<a href="https://colab.research.google.com/github/srimanrishi/Outlier-Detection-and-Handling-in-Datasets-using-Python/blob/main/outlier_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/preprocessing_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/preprocessing_dataset.csv'

In [ ]:
import pandas as pd
import numpy as np

# Create dummy data for demonstration
data = {
    'Age': [25, 30, np.nan, 35, 40, 22, 28, 60, 29, 31, 15, 50, 30, 27, 45],
    'Salary': [50000, 60000, 75000, np.nan, 80000, 45000, 55000, 150000, 62000, 68000, 30000, 90000, 70000, 52000, 85000],
    'Country': ['USA', 'Canada', 'USA', 'Germany', 'France', 'USA', 'UK', 'USA', 'Germany', 'Canada', 'France', 'UK', 'USA', 'Canada', 'Germany'],
    'Purchased': ['No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No']
}
df_dummy = pd.DataFrame(data)

# Save the dummy DataFrame to a CSV file
df_dummy.to_csv('/content/preprocessing_dataset.csv', index=False)
print("Dummy dataset 'preprocessing_dataset.csv' created successfully.")

Dummy dataset 'preprocessing_dataset.csv' created successfully.


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
!pip install missingno

In [ ]:
import missingno as msno
msno.bar(df)

In [ ]:
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')

In [ ]:
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Salary'] = df['Salary'].fillna(df['Salary'].mean())

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
df[['Age']] = imputer.fit_transform(df[['Age']])
df

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

num_features = df.select_dtypes(include=['int64', 'float64']).columns
cat_features = df.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features)
    ])
df_transformed = preprocessor.fit_transform(df)
df1 = pd.DataFrame(df_transformed.toarray(), columns=preprocessor.get_feature_names_out())
df1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

plt.figure(figsize=(15, 6))
for i, col in enumerate(numerical_cols):
    plt.subplot(1, len(numerical_cols), i + 1)
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot of {col}')
    plt.ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = df[(df['Age'] < lower_bound) | (df['Age'] > upper_bound)]
outliers

In [ ]:
from scipy.stats import zscore
z_scores = zscore(df.select_dtypes(include=np.number))
df[np.abs(z_scores) > 2.5]

In [ ]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.1)
df['outlier'] = iso.fit_predict(df[['Age','Salary']])
df[df['outlier'] == -1]

In [ ]:
outlier_indices = outliers.index
df1_cleaned = df1.drop(index=outlier_indices)
df1_cleaned